In [1]:
from glob import glob
from shutil import copyfile
import os

import numpy as np
import pandas as pd

from keras.layers.core import Dense, Dropout
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers.core import *

from fastai.vgg16bn import Vgg16BN
from fastai.utils import *

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


# Setup Files

In [7]:
%cd statefarm
# %mkdir valid
# %mkdir results
# %mkdir -p sample/train
# %mkdir -p sample/test
# %mkdir -p sample/valid
# %mkdir -p sample/results

/home/ubuntu/projects/statefarm


In [12]:
%cd train

/home/ubuntu/projects/statefarm/train


In [13]:
g = glob('*')
for d in g: 
#     os.mkdir('../sample/train/'+d)
#     os.mkdir('../sample/valid/'+d)
    os.mkdir('../valid/'+d)
%ls ../sample/train

c0/  c1/  c2/  c3/  c4/  c5/  c6/  c7/  c8/  c9/


In [ ]:
g = glob('*/*.jpg')
sample_size = int(len(g) * .25)
shuf = np.random.permutation(g)
for i in range(sample_size): copyfile(shuf[i], '../sample/train/' + shuf[i])

In [ ]:
g = glob('*/*.jpg')
valid_size = int(len(g) * .1)
shuf = np.random.permutation(g)
for i in range(valid_size): copyfile(shuf[i], '../sample/valid/' + shuf[i])

In [14]:
%cd ../..

/home/ubuntu/projects


In [15]:
%cd statefarm

/home/ubuntu/projects/statefarm


Select a couple of subjects for validation

In [16]:
imgs = pd.read_csv('driver_imgs_list.csv')
print imgs.head()
subjects = imgs['subject'].unique()
valid_set = imgs[imgs['subject'].isin(subjects[:3])]
print valid_set.shape

def copy_img(r):
    p = os.path.join(r['classname'], r['img'])
    os.rename('train/' + p, 'valid/' + p)

valid_set.apply(copy_img, axis=1)

  subject classname            img
0    p002        c0  img_44733.jpg
1    p002        c0  img_72999.jpg
2    p002        c0  img_25094.jpg
3    p002        c0  img_69092.jpg
4    p002        c0  img_92629.jpg
(2424, 3)


0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
        ... 
2394    None
2395    None
2396    None
2397    None
2398    None
2399    None
2400    None
2401    None
2402    None
2403    None
2404    None
2405    None
2406    None
2407    None
2408    None
2409    None
2410    None
2411    None
2412    None
2413    None
2414    None
2415    None
2416    None
2417    None
2418    None
2419    None
2420    None
2421    None
2422    None
2423    None
dtype: object

In [17]:
g = glob('valid/*/*.jpg')
len(g)

2424

In [18]:
%cd ..

/home/ubuntu/projects


# Load Images

In [2]:
sample_path = 'statefarm/sample/'
full_path = 'statefarm/'
path = full_path

In [3]:
gen = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)

train = get_batches(path+'train', gen)
valid = get_batches(path+'valid')
train_labels = onehot(train.classes)
valid_labels = onehot(valid.classes)

Found 20000 images belonging to 10 classes.
Found 2424 images belonging to 10 classes.


In [3]:
vgg = Vgg16BN()
vgg.model.pop()
vgg.model.add(Dense(train.nb_class, activation='softmax'))
for layer in vgg.model.layers[:-7]:
    layer.trainable = False
vgg.compile(lr=0.00005)

NameError: name 'train' is not defined

In [29]:
vgg = Vgg16BN()
vgg_conv = Sequential(vgg.model.layers[:-9])
keras.__version__

'1.1.0'

In [31]:
train_conv_feat = vgg_conv.predict_generator(train, train.nb_sample*5)
train_conv_labels = np.concatenate([train_labels]*5)

In [33]:
valid_conv_feat = vgg_conv.predict_generator(valid, valid.nb_sample)

In [34]:
save_array(full_path + 'train_conv_feat.dat', train_conv_feat)
save_array(full_path + 'train_conv_labels.dat', train_conv_labels)
save_array(full_path + 'valid_conv_feat.dat', valid_conv_feat)

In [4]:
train_conv_feat = load_array(full_path + 'train_conv_feat.dat')
train_conv_labels = load_array(full_path + 'train_conv_labels.dat')
valid_conv_feat = load_array(full_path + 'valid_conv_feat.dat')

In [27]:
batch_size = len(train_conv_feat)/5
for i in range(0, 5):
    save_array(full_path + 'train_conv_feat{}.dat'.format(i+1), train_conv_feat[batch_size*i:batch_size*(i+1)])

In [22]:
import gc
gc.collect()
vgg = Vgg16BN(include_top=False)
vgg.model.summary()
# def get_dense_layers(p):
#     return [
#         MaxPooling2D(input_shape=vgg_conv.layers[-1].output_shape[1:]),
#         Flatten(),
#         Dense(128, activation='relu'),
#         BatchNormalization(),
#         Dropout(p),
#         Dense(128, activation='relu'),
#         BatchNormalization(),
#         Dropout(p),
#         Dense(10, activation='softmax')
#         ]
# p = 0.5
# vgg_den = Sequential()
# vgg_den.add(MaxPooling2D(input_shape=vgg.model.layers[-1].output_shape[1:]))
# vgg_den.add(Flatten())
# vgg_den.add(Dense(128, activation='relu'))
# vgg_den.add(BatchNormalization())
# vgg_den.add(Dropout(p))
# vgg_den.add(Dense(128, activation='relu'))
# vgg_den.add(BatchNormalization())
# vgg_den.add(Dropout(p))
# vgg_den.add(Dense(10, activation='softmax'))
# vgg_den.summary()
# vgg_den.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

Problem occurred during compilation with the command line below:
/usr/bin/g++ -shared -g -O3 -fno-math-errno -Wno-unused-label -Wno-unused-variable -Wno-write-strings -DNPY_NO_DEPRECATED_API=NPY_1_7_API_VERSION -m64 -fPIC -I/home/ubuntu/anaconda2/lib/python2.7/site-packages/numpy/core/include -I/home/ubuntu/anaconda2/include/python2.7 -I/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/gof -fvisibility=hidden -o /home/ubuntu/.theano/compiledir_Linux-4.4--generic-x86_64-with-debian-stretch-sid-x86_64-2.7.12-64/tmpi0VldC/a48f4668d98d8c16519ad508b2f7c269.so /home/ubuntu/.theano/compiledir_Linux-4.4--generic-x86_64-with-debian-stretch-sid-x86_64-2.7.12-64/tmpi0VldC/mod.cpp -L/home/ubuntu/anaconda2/lib -lpython2.7
ERROR (theano.gof.cmodule): [Errno 12] Cannot allocate memory


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_16 (Lambda)               (None, 3, 224, 224)   0           lambda_input_16[0][0]            
____________________________________________________________________________________________________
zeropadding2d_196 (ZeroPadding2D)(None, 3, 226, 226)   0           lambda_16[0][0]                  
____________________________________________________________________________________________________


OSError: [Errno 12] Cannot allocate memory

In [51]:
vgg_den.fit(train_conv_feat, train_conv_labels, batch_size=64, nb_epoch=1, 
             validation_data=(valid_conv_feat, valid_labels))

Problem occurred during compilation with the command line below:
/usr/bin/g++ -shared -g -O3 -fno-math-errno -Wno-unused-label -Wno-unused-variable -Wno-write-strings -march=broadwell -mmmx -mno-3dnow -msse -msse2 -msse3 -mssse3 -mno-sse4a -mcx16 -msahf -mmovbe -maes -mno-sha -mpclmul -mpopcnt -mabm -mno-lwp -mfma -mno-fma4 -mno-xop -mbmi -mbmi2 -mno-tbm -mavx -mavx2 -msse4.2 -msse4.1 -mlzcnt -mrtm -mhle -mrdrnd -mf16c -mfsgsbase -mno-rdseed -mno-prfchw -mno-adx -mfxsr -mxsave -mxsaveopt -mno-avx512f -mno-avx512er -mno-avx512cd -mno-avx512pf -mno-prefetchwt1 -mno-clflushopt -mno-xsavec -mno-xsaves -mno-avx512dq -mno-avx512bw -mno-avx512vl -mno-avx512ifma -mno-avx512vbmi -mno-clwb -mno-pcommit -mno-mwaitx --param l1-cache-size=32 --param l1-cache-line-size=64 --param l2-cache-size=46080 -mtune=broadwell -DNPY_NO_DEPRECATED_API=NPY_1_7_API_VERSION -m64 -fPIC -I/home/ubuntu/anaconda2/lib/python2.7/site-packages/numpy/core/include -I/home/ubuntu/anaconda2/include/python2.7 -I/home/ubuntu/a

OSError: [Errno 12] Cannot allocate memory

# Train

In [ ]:
vgg.fit(train, valid)

In [ ]:
vgg.model.save_weights(full_path + 'statefarm.1.h5')

In [ ]:
vgg.compile(lr=0.00001)
vgg.fit(train, valid)

In [ ]:
vgg.model.save_weights(full_path + 'statefarm.2.h5')

In [ ]:
vgg.compile(lr=0.0000001)
vgg.fit(train, valid)

In [ ]:
vgg.model.save_weights(full_path + 'statefarm.3.h5')

In [30]:
vgg.model.load_weights(full_path + 'statefarm.3.h5')

In [31]:
vgg.model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  0           zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
___________________________________________________________________________________________

In [32]:
def proc_wgts(layer): return [o/2 for o in layer.get_weights()]

for layer in vgg.model.layers[-7:]:
    if type(layer) is Dropout:
        layer.rate = 0
    else:
        proc_wgts(layer)

In [33]:
vgg.compile(lr=0.00000001)
vgg.fit(train, valid)

Epoch 1/1
20182/20182 [==============================] - 971s - loss: 0.7809 - acc: 0.8211 - val_loss: 0.1660 - val_acc: 0.9790


In [34]:
vgg.model.save_weights(full_path + 'statefarm.4.h5')

In [35]:
vgg.compile(lr=0.000000001)
vgg.fit(train, valid)

Epoch 1/1
20182/20182 [==============================] - 970s - loss: 0.7855 - acc: 0.8197 - val_loss: 0.1811 - val_acc: 0.9773


In [70]:
vgg.model.load_weights(full_path + 'statefarm.4.h5')

In [5]:
batches, preds = vgg.test(path + 'test', batch_size=128)
print path + 'test'
print batches.filenames[:5]
print preds[:5]

Found 79726 images belonging to 1 classes.
statefarm/test
['cx/img_42265.jpg', 'cx/img_56819.jpg', 'cx/img_44646.jpg', 'cx/img_37236.jpg', 'cx/img_43344.jpg']
[[  3.17076010e-09   1.00675572e-10   1.92204283e-07   9.99435008e-01
    2.98874540e-04   5.56419982e-05   6.84871168e-07   1.15514049e-05
    1.97905669e-04   1.45766194e-07]
 [  1.81783107e-03   8.79171491e-03   1.78203620e-02   1.64828543e-02
    2.73116399e-04   7.75305787e-03   6.18974790e-02   6.27453253e-02
    7.46752560e-01   7.56656080e-02]
 [  9.95868802e-01   2.76599998e-14   3.98979842e-11   4.80411826e-13
    3.51028234e-11   6.27098165e-11   6.04716210e-17   3.03397030e-11
    3.95617938e-09   4.13118163e-03]
 [  6.93303137e-10   1.77033089e-05   2.51719845e-09   3.73918424e-10
    1.53531632e-10   4.64906122e-07   3.07112545e-01   2.02661496e-07
    6.92868769e-01   3.01028251e-07]
 [  1.12341167e-08   7.10699144e-09   8.96826711e-15   6.22740227e-12
    4.80660520e-12   9.99993563e-01   4.68911802e-13   8.646317

In [6]:
from fastai import utils

utils.save_array(full_path + 'preds.dat', preds)
utils.save_array(full_path + 'filenames.dat', batches.filenames)

/home/ubuntu/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [55]:
import pandas as pd
#subm = np.hstack((np.array(batches.filenames), np.array(preds)))
filenames = [ os.path.basename(f) for f in batches.filenames ]
subm = pd.DataFrame(preds, index=filenames, columns=['c0','c1','c2','c3','c4','c5','c6','c7','c8','c9'])
subm.index.name = 'img'
subm.to_csv(full_path + 'submission1.csv')

In [56]:
from IPython.display import FileLink
FileLink(full_path + 'submission1.csv')

/home/ubuntu/projects/statefarm/submission1.csv

**Scores:**
2.26947, 2.75651

In [64]:
for layer in vgg.model.layers:
    layer.trainable = True

In [ ]:
vgg.compile(lr=0.0000001)
vgg.fit(train, valid)

Epoch 1/1
 2168/20182 [==>...........................] - ETA: 1820s - loss: 1.9768 - acc: 0.6776